<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/125_using_example_cde_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Using Earth engine examples**


- ee datasets seem to come from https://code.earthengine.google.com/scripts/load
- When logged into gee, this link returns a f.json. It contains all code examples from the docs, including the datasets.
- community datasets can be accessed using git from https://earthengine.googlesource.com
- which in many cases can be translated into python code
- this should make it feasible to use the code examples directly into geemap

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install -U geemap

In [ ]:
import ee
import geemap
import json

## Manually download f.json 
https://code.earthengine.google.com/scripts/load

## Explore json contents

Change file download location. Also added f.json to repo.

In [ ]:
url = "https://github.com/gee-community/geemap/blob/master/geemap/data/gee_f.json"
geemap.download_file(url)

In [ ]:
with open("gee_f.json", encoding="utf-8") as f:
    functions = json.load(f)

functions.keys()

In [ ]:
[(x["name"], len(x["contents"])) for x in functions["examples"]]

## Details

In [ ]:
details = [
    (dataset["name"], dataset["code"])
    for x in functions["examples"]
    for dataset in x["contents"]
    if x["name"] == "Datasets"
]
details[0]

## Convert js to py

In [ ]:
def get_py(js):
    try:
        return geemap.js_snippet_to_py(
            js, add_new_cell=False, import_ee=False, import_geemap=False, show_map=False
        )
    except:
        return None


results_convert = [get_py(js) for _, js in details]

In [ ]:
len(details), len(results_convert)

Except for 2 cases, converts ran without error.

In [ ]:
len([x for x in results_convert if x])

## Verify availability

In [ ]:
from geemap import datasets

len(datasets.get_ee_stac_list())

In [ ]:
known_datasets = set(name.replace("/", "_") for name in datasets.get_ee_stac_list())
known_code = set(name for name, _ in details)

## Only some have exact matches

In [ ]:
len(known_code & known_datasets)

Sometimes there is a 2 example available for a dataset, mostly FeatureViews.

In [ ]:
excess = known_code - known_datasets

In [ ]:
len(excess)

In [ ]:
excess_no_featureview = set(x.replace("_FeatureView", "") for x in known_code)
len(excess_no_featureview - known_datasets)

Leftovers

In [ ]:
(excess_no_featureview - known_datasets)

## Datasets without examples

In [ ]:
missing_code = known_datasets - known_code
len(missing_code)

These are also missing in gee itself (logically as these load from f.json). 

- https://code.earthengine.google.com/?scriptPath=Examples%3ADatasets%2FLANDSAT_LC8

Deprecated stuff afaik, s.a. Landsat LT1

In [ ]:
missing_code

In [ ]:
from geemap import common

[x["title"] for x in common.search_ee_data("LANDSAT L1T")][:5]

In [ ]:
[(x["uid"], x["title"]) for x in common.search_ee_data("MODIS MYD")][:5]